In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline

import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
if torch.cuda.is_available():
    torch.cuda.caching_allocator_delete
    torch.cuda.empty_cache()  # 释放显存
    print('Memory Allocated', torch.cuda.memory_allocated() )

# 模型转到gpu： model.to(device)
# 训练数据转移到gpu上: x, y = x.to(device), y.to(device) ; 训练数据和test数据都都要


In [ ]:
# torch.cuda.memory_stats()
# torch.cuda.memory_summary()
# torch.cuda.memory_allocated()

In [ ]:
# 数据预处理
# df = pd.read_csv("../dataset/creditcard-tianchi.csv", index_col=None)
# del df['Time']
# df.head(3)
# df.head(30000).to_csv("../dataset/creditcard-3w.csv", index=False)  # 分拆数据到 creditcard-3w
# df.to_csv("../dataset/creditcard-all.csv", index=False)  # 分拆数据到 creditcard-3w

In [ ]:
# 读取数据
df = pd.read_csv("../dataset/creditcard-all.csv", index_col=None)
# del df['Unnamed: 0']
df.head(3),df.info()

In [ ]:
df.Class.unique(), df.Class.value_counts()

In [ ]:
# 列处理2
# df.groupby(['salary', 'sales']).size()

# pd.get_dummies(df.salary) # field内容编程列 list
# df = df.join( pd.get_dummies(df.salary) ) # 把列加入到 pandas fields
# del df['salary'] # 删除原来列

# pd.get_dummies(df.sales)  # 读日编码？？？
# df = df.rename(columns={'sales':'department'})  # 改列名
# df = df.join(pd.get_dummies(df.department))
# del df['department']
# df.head(3).to_csv("../dataset/test1.csv", index=False)

In [ ]:

df.shape  # 行列？
df.size # df的行x列， 共有多少数据cell
df.Class.unique(), df.Class.value_counts()
# df.columns

In [ ]:
Y_data = df.Class.values.reshape(-1,1)  # 转换成 pd type array
Y_data
# Y = torch.from_numpy(Y_data).type(torch.float32)  # 转换成tensor
# Y = torch.from_numpy(Y_data).type(torch.FloatTensor).to(device)
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
Y.size()

In [ ]:
X_columns = [c for c in df.columns if c != 'Class']
X_data = df[X_columns]  
# X = torch.from_numpy(X_data.values).type(torch.float32).to(device)
X = torch.from_numpy(X_data.values).type(torch.float32)
X.shape, X.size()

In [ ]:
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用
class Model(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
lr = 0.0001
def get_model():
    model = Model()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

In [ ]:
model, optim = get_model()
model = model.to(device)
print( model, "\n" )
print( optim )
print(next(model.parameters()).device) # 应该打印 cuda:0

In [ ]:
loss_fn = nn.BCELoss()
# loss_fn = nn.BCELoss().to(device)
# batch = 64
# batch = 4096
# batch = 8192
# batch = 16384
batch = 32768 # 一次性加载
no_of_batches = len(X)//batch
epochs = 10
from torch.utils.data import TensorDataset, DataLoader


In [ ]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# from torch.utils.data import TensorDataset, DataLoader
# HR_dataset = TensorDataset(X, Y)
# train_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

In [ ]:
# Ver 4.2 - sklearn mode
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32).to(device)
test_y = torch.from_numpy(test_y).type(torch.float32).to(device)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [ ]:
%%time


# epochs = 10
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

In [ ]:
# 最后一次计算结果：

epoch_accuracy = accuracy(model(train_x), train_y)
epoch_loss = loss_fn(model(train_x), train_y).data.item()
epoch_test_accuracy = accuracy(model(test_x), test_y)
epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
print('epoch: ', 
    ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
    ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

In [ ]:
# print( loss_fn(model(train_x), train_y).data.item() )
# accuracyRate = ((model(train_x).data.numpy()).astype('int') == train_y.numpy().astype('int')).mean()
# print("训练数据 - 准确率%：", accuracyRate*100) 

# print( loss_fn(model(test_x), test_y).data.item() )
# accuracyRate = ((model(test_x).data.numpy()).astype('int') == test_y.numpy().astype('int')).mean()
# print("测试数据 - 准确率%：", accuracyRate*100) 

- 数据准确性验证
- 过拟合/欠拟合
    - 过拟合：对于已知数据过度拟合，对未知数据预测性差
    - 欠拟合：对于已知数据拟合不够，对未知数据预测性差